In [ ]:
import os
import cv2
import numpy as np
import random
import glob
import shutil
from typing import List, Tuple, Optional, Union
from pathlib import Path

class DataAugmentation:
    """
    Class for augmenting image datasets with corresponding YOLO format annotations.
    
    This class performs various augmentations on images while properly transforming
    their corresponding YOLO format annotations (class_id, x_center, y_center, width, height).
    """
    
    def __init__(self, source_dir: str, target_dir: str) -> None:
        """
        Initialize the DataAugmentation class.
        
        Args:
            source_dir: Directory containing 'images' and 'labels' folders
            target_dir: Directory where augmented images and labels will be saved
        """
        self.source_dir = source_dir
        self.target_dir = target_dir
        
        # Define source and target subdirectories
        self.images_dir = os.path.join(source_dir, "images")
        self.labels_dir = os.path.join(source_dir, "labels")
        self.target_images_dir = os.path.join(target_dir, "images")
        self.target_labels_dir = os.path.join(target_dir, "labels")
        
        # Create target directories if they don't exist
        os.makedirs(self.target_images_dir, exist_ok=True)
        os.makedirs(self.target_labels_dir, exist_ok=True)
    
    def load_data(self) -> List[Tuple[str, str]]:
        """Load image paths and their corresponding label paths."""
        image_paths = glob.glob(os.path.join(self.images_dir, "*.jpg")) + \
                      glob.glob(os.path.join(self.images_dir, "*.jpeg")) + \
                      glob.glob(os.path.join(self.images_dir, "*.png"))
        
        image_label_pairs = []
        for img_path in image_paths:
            base_name = os.path.basename(img_path)
            name_without_ext = os.path.splitext(base_name)[0]
            label_path = os.path.join(self.labels_dir, f"{name_without_ext}.txt")
            
            if os.path.exists(label_path):
                image_label_pairs.append((img_path, label_path))
        
        return image_label_pairs
    
    def load_labels(self, label_path: str) -> np.ndarray:
        """Load YOLO format labels from a file."""
        labels = []
        with open(label_path, 'r') as f:
            for line in f:
                values = line.strip().split()
                if len(values) == 5:
                    class_id = int(values[0])
                    x_center = float(values[1])
                    y_center = float(values[2])
                    width = float(values[3])
                    height = float(values[4])
                    labels.append([class_id, x_center, y_center, width, height])
        return np.array(labels)
    
    def save_labels(self, label_path: str, labels: np.ndarray) -> None:
        """Save labels in YOLO format to a file."""
        with open(label_path, 'w') as f:
            for label in labels:
                class_id, x_center, y_center, width, height = label
                f.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
    
    # Geometric Transformations
    
    def rotate(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Rotate image and update labels."""
        h, w = image.shape[:2]
        angle = random.uniform(-30, 30)
        
        # Rotate image
        center = (w // 2, h // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)
        
        if len(labels) == 0:
            return rotated_image, labels
        
        rotated_labels = []
        for label in labels:
            class_id, x_center, y_center, width, height = label
            
            # Convert normalized coordinates to pixel
            x_center_px = x_center * w
            y_center_px = y_center * h
            width_px = width * w
            height_px = height * h
            
            # Get the four corners of the bounding box
            half_w, half_h = width_px/2, height_px/2
            corners = np.array([
                [x_center_px - half_w, y_center_px - half_h, 1],
                [x_center_px + half_w, y_center_px - half_h, 1],
                [x_center_px + half_w, y_center_px + half_h, 1],
                [x_center_px - half_w, y_center_px + half_h, 1]
            ])
            
            # Apply rotation to corners
            rotated_corners = np.dot(rotation_matrix, corners.T).T
            
            # Get bounding box of rotated corners
            min_x = max(0, np.min(rotated_corners[:, 0]))
            min_y = max(0, np.min(rotated_corners[:, 1]))
            max_x = min(w, np.max(rotated_corners[:, 0]))
            max_y = min(h, np.max(rotated_corners[:, 1]))
            
            # Convert back to YOLO format (normalized)
            new_x_center = (min_x + max_x) / 2 / w
            new_y_center = (min_y + max_y) / 2 / h
            new_width = (max_x - min_x) / w
            new_height = (max_y - min_y) / h
            
            # Check if box is valid
            if new_width > 0 and new_height > 0:
                rotated_labels.append([class_id, new_x_center, new_y_center, new_width, new_height])
        
        return rotated_image, np.array(rotated_labels)
    
    def flip(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Apply horizontal or vertical flip."""
        # Preferring horizontal flips for climbing scenarios (more realistic)
        if random.random() < 0.8:  # 80% chance of horizontal flip
            flipped_image = cv2.flip(image, 1)
            flipped_labels = []
            
            for label in labels:
                class_id, x_center, y_center, width, height = label
                # For horizontal flip, only x_center changes
                flipped_labels.append([class_id, 1.0 - x_center, y_center, width, height])
        else:  # 20% chance of vertical flip
            flipped_image = cv2.flip(image, 0)
            flipped_labels = []
            
            for label in labels:
                class_id, x_center, y_center, width, height = label
                # For vertical flip, only y_center changes
                flipped_labels.append([class_id, x_center, 1.0 - y_center, width, height])
        
        return flipped_image, np.array(flipped_labels)
    
    def scale(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Scale image and update labels."""
        h, w = image.shape[:2]
        scale_x = random.uniform(0.8, 1.2)
        scale_y = random.uniform(0.8, 1.2)
        
        # Scale image
        scaled_image = cv2.resize(image, None, fx=scale_x, fy=scale_y, interpolation=cv2.INTER_LINEAR)
        new_h, new_w = scaled_image.shape[:2]
        
        # Resize back to original size
        scaled_image = cv2.resize(scaled_image, (w, h), interpolation=cv2.INTER_LINEAR)
        
        # Update labels
        scale_factor_x = scale_x * (w / new_w)
        scale_factor_y = scale_y * (h / new_h)
        
        scaled_labels = []
        for label in labels:
            class_id, x_center, y_center, width, height = label
            
            # Apply scaling
            new_x_center = 0.5 + (x_center - 0.5) / scale_factor_x
            new_y_center = 0.5 + (y_center - 0.5) / scale_factor_y
            new_width = width / scale_factor_x
            new_height = height / scale_factor_y
            
            # Clip values to [0, 1]
            new_x_center = np.clip(new_x_center, 0, 1)
            new_y_center = np.clip(new_y_center, 0, 1)
            new_width = np.clip(new_width, 0, 1)
            new_height = np.clip(new_height, 0, 1)
            
            scaled_labels.append([class_id, new_x_center, new_y_center, new_width, new_height])
        
        return scaled_image, np.array(scaled_labels)
    
    def translate(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Translate image and update labels."""
        h, w = image.shape[:2]
        
        # Maximum translation is 20% of the image dimensions
        tx = random.uniform(-0.2, 0.2) * w
        ty = random.uniform(-0.2, 0.2) * h
        
        # Translation matrix
        trans_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
        translated_image = cv2.warpAffine(image, trans_matrix, (w, h))
        
        # Update labels
        translated_labels = []
        for label in labels:
            class_id, x_center, y_center, width, height = label
            
            # Apply translation (normalized)
            new_x_center = x_center + tx / w
            new_y_center = y_center + ty / h
            
            # Check if center is still within image bounds
            if 0 <= new_x_center <= 1 and 0 <= new_y_center <= 1:
                translated_labels.append([class_id, new_x_center, new_y_center, width, height])
        
        return translated_image, np.array(translated_labels)
    
    def crop_and_resize(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Randomly crop image and resize to original dimensions."""
        h, w = image.shape[:2]
        
        # Crop between 60-90% of original size
        crop_ratio = random.uniform(0.6, 0.9)
        crop_w = int(w * crop_ratio)
        crop_h = int(h * crop_ratio)
        
        # Random crop position
        x1 = random.randint(0, w - crop_w)
        y1 = random.randint(0, h - crop_h)
        x2 = x1 + crop_w
        y2 = y1 + crop_h
        
        # Crop image
        cropped_image = image[y1:y2, x1:x2]
        
        # Resize back to original dimensions
        resized_image = cv2.resize(cropped_image, (w, h), interpolation=cv2.INTER_LINEAR)
        
        # Update labels
        cropped_labels = []
        for label in labels:
            class_id, x_center, y_center, width, height = label
            
            # Convert to absolute pixel coordinates
            abs_x_center = x_center * w
            abs_y_center = y_center * h
            abs_width = width * w
            abs_height = height * h
            
            # Check if bounding box is within crop
            bbox_x1 = abs_x_center - abs_width / 2
            bbox_y1 = abs_y_center - abs_height / 2
            bbox_x2 = abs_x_center + abs_width / 2
            bbox_y2 = abs_y_center + abs_height / 2
            
            # Calculate intersection
            inter_x1 = max(bbox_x1, x1)
            inter_y1 = max(bbox_y1, y1)
            inter_x2 = min(bbox_x2, x2)
            inter_y2 = min(bbox_y2, y2)
            
            # Skip if no intersection
            if inter_x1 >= inter_x2 or inter_y1 >= inter_y2:
                continue
            
            # New bounding box in cropped coordinates
            new_bbox_x1 = (inter_x1 - x1) / crop_w
            new_bbox_y1 = (inter_y1 - y1) / crop_h
            new_bbox_x2 = (inter_x2 - x1) / crop_w
            new_bbox_y2 = (inter_y2 - y1) / crop_h
            
            # Convert back to YOLO format
            new_x_center = (new_bbox_x1 + new_bbox_x2) / 2
            new_y_center = (new_bbox_y1 + new_bbox_y2) / 2
            new_width = new_bbox_x2 - new_bbox_x1
            new_height = new_bbox_y2 - new_bbox_y1
            
            cropped_labels.append([class_id, new_x_center, new_y_center, new_width, new_height])
        
        return resized_image, np.array(cropped_labels)
    
    # Photometric Transformations
    
    def adjust_brightness(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Adjust image brightness."""
        # Convert to HSV for easier brightness manipulation
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        # Adjust V channel (brightness)
        factor = random.uniform(0.5, 1.5)
        hsv[:, :, 2] = np.clip(hsv[:, :, 2] * factor, 0, 255).astype(np.uint8)
        
        # Convert back to BGR
        adjusted_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        # Labels unchanged
        return adjusted_image, labels
    
    def adjust_contrast(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Adjust image contrast."""
        factor = random.uniform(0.5, 1.5)
        mean = np.mean(image, axis=(0, 1), keepdims=True)
        adjusted_image = np.clip((image - mean) * factor + mean, 0, 255).astype(np.uint8)
        
        # Labels unchanged
        return adjusted_image, labels
    
    def color_jitter(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Randomly alter hue, saturation, and color balance."""
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        # Adjust hue
        hue_shift = random.uniform(-10, 10)
        hsv[:, :, 0] = np.clip(hsv[:, :, 0] + hue_shift, 0, 179).astype(np.uint8)
        
        # Adjust saturation
        sat_factor = random.uniform(0.5, 1.5)
        hsv[:, :, 1] = np.clip(hsv[:, :, 1] * sat_factor, 0, 255).astype(np.uint8)
        
        # Convert back to BGR
        adjusted_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        # Random channel shifts
        for i in range(3):  # BGR channels
            adjusted_image[:, :, i] = np.clip(
                adjusted_image[:, :, i] * random.uniform(0.9, 1.1), 0, 255
            ).astype(np.uint8)
        
        # Labels unchanged
        return adjusted_image, labels
    
    def add_noise(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Add Gaussian noise to the image."""
        mean = 0
        std = random.uniform(5, 20)
        noise = np.random.normal(mean, std, image.shape).astype(np.int16)
        noisy_image = np.clip(image.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        
        # Labels unchanged
        return noisy_image, labels
    
    def blur_sharpen(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Apply Gaussian blur or sharpening filter."""
        if random.random() < 0.5:  # Blur
            kernel_size = random.choice([3, 5, 7])
            processed_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
        else:  # Sharpen using unsharp mask
            blur = cv2.GaussianBlur(image, (5, 5), 3)
            processed_image = cv2.addWeighted(image, 1.5, blur, -0.5, 0)
            
        # Labels unchanged
        return processed_image, labels
    
    # Advanced Augmentation
    
    def random_erase(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Randomly mask out sections of the image."""
        h, w = image.shape[:2]
        erased_image = image.copy()
        
        # Number of rectangles to erase
        num_rectangles = random.randint(1, 3)
        
        for _ in range(num_rectangles):
            # Size between 5-15% of image
            rect_w = int(random.uniform(0.05, 0.15) * w)
            rect_h = int(random.uniform(0.05, 0.15) * h)
            
            # Random position
            x1 = random.randint(0, w - rect_w)
            y1 = random.randint(0, h - rect_h)
            
            # Fill with random color or mean pixel value
            if random.random() < 0.5:
                color = np.random.randint(0, 255, 3).tolist()
            else:
                color = list(map(int, np.mean(image, axis=(0, 1))))
            
            erased_image[y1:y1+rect_h, x1:x1+rect_w] = color
        
        # Labels unchanged
        return erased_image, labels
    
    def mixup(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Combine two images and their labels."""
        # Load all image-label pairs
        all_pairs = self.load_data()
        
        # If no other images available, return original
        if len(all_pairs) <= 1:
            return image, labels
        
        # Select random second image
        second_img_path, second_label_path = random.choice(all_pairs)
        second_img = cv2.imread(second_img_path)
        
        # Ensure compatible dimensions
        if second_img is None or second_img.shape != image.shape:
            return image, labels
            
        second_labels = self.load_labels(second_label_path)
        
        # Mix weight
        alpha = random.uniform(0.2, 0.8)
        
        # Mix images
        mixed_image = cv2.addWeighted(image, alpha, second_img, 1 - alpha, 0)
        
        # Combine labels (keeping all from both images)
        mixed_labels = np.vstack([labels, second_labels]) if len(labels) > 0 and len(second_labels) > 0 else \
                      labels if len(labels) > 0 else second_labels
        
        return mixed_image, mixed_labels
    
    def perspective_transform(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Apply perspective warps to simulate different viewpoints."""
        h, w = image.shape[:2]
        
        # Get perspective transformation matrix
        magnitude = 0.05  # Max distortion as fraction of image dimensions
        src_pts = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
        dst_pts = np.float32([
            [random.uniform(0, magnitude * w), random.uniform(0, magnitude * h)],  # top-left
            [random.uniform(w * (1 - magnitude), w), random.uniform(0, magnitude * h)],  # top-right
            [random.uniform(w * (1 - magnitude), w), random.uniform(h * (1 - magnitude), h)],  # bottom-right
            [random.uniform(0, magnitude * w), random.uniform(h * (1 - magnitude), h)]  # bottom-left
        ])
        
        perspective_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
        warped_image = cv2.warpPerspective(image, perspective_matrix, (w, h))
        
        # Update labels (this is complex, we'll simplify by transforming box corners)
        warped_labels = []
        for label in labels:
            class_id, x_center, y_center, width, height = label
            
            # Convert to absolute pixel coordinates 
            x_center_px = x_center * w
            y_center_px = y_center * h
            width_px = width * w
            height_px = height * h
            
            # Get the four corners
            corners = np.array([
                [x_center_px - width_px/2, y_center_px - height_px/2, 1],
                [x_center_px + width_px/2, y_center_px - height_px/2, 1],
                [x_center_px + width_px/2, y_center_px + height_px/2, 1],
                [x_center_px - width_px/2, y_center_px + height_px/2, 1]
            ])
            
            # Transform corners
            warped_corners = []
            for corner in corners:
                # Apply perspective transformation
                pt = np.dot(perspective_matrix, corner)
                # Normalize by third coordinate
                pt = pt / pt[2] if pt[2] != 0 else pt
                warped_corners.append(pt[:2])
            
            warped_corners = np.array(warped_corners)
            
            # Get new bounding box
            min_x = max(0, np.min(warped_corners[:, 0]))
            min_y = max(0, np.min(warped_corners[:, 1]))
            max_x = min(w, np.max(warped_corners[:, 0]))
            max_y = min(h, np.max(warped_corners[:, 1]))
            
            # Convert back to YOLO format
            new_x_center = (min_x + max_x) / 2 / w
            new_y_center = (min_y + max_y) / 2 / h
            new_width = (max_x - min_x) / w
            new_height = (max_y - min_y) / h
            
            # Check if box is valid
            if 0 < new_width < 1 and 0 < new_height < 1:
                warped_labels.append([class_id, new_x_center, new_y_center, new_width, new_height])
        
        return warped_image, np.array(warped_labels)
    
    def mosaic(self, image: np.ndarray, labels: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Combine four images into one using mosaic augmentation."""
        # Load all image-label pairs
        all_pairs = self.load_data()
        
        # Need at least 3 images for mosaic (current + 3 more)
        if len(all_pairs) < 4:
            return image, labels
        
        # Original image dimensions
        h, w = image.shape[:2]
        
        # Create mosaic canvas
        mosaic_img = np.zeros((h, w, 3), dtype=np.uint8)
        
        # Select center point (with jitter)
        center_x = int(random.uniform(0.4, 0.6) * w)
        center_y = int(random.uniform(0.4, 0.6) * h)
        
        # Select 3 additional random images
        additional_images = random.sample(all_pairs, 3)
        all_images = [(image, labels)] + [(cv2.imread(img_path), self.load_labels(label_path)) 
                                         for img_path, label_path in additional_images]
        
        # Ensure all images loaded correctly
        all_images = [(img, lbl) for img, lbl in all_images if img is not None and img.shape[:2] == (h, w)]
        if len(all_images) < 4:
            return image, labels
            
        # Place images in quadrants
        positions = [
            # xmin, ymin, xmax, ymax
            [0, 0, center_x, center_y],               # top-left
            [center_x, 0, w, center_y],               # top-right
            [0, center_y, center_x, h],               # bottom-left
            [center_x, center_y, w, h]                # bottom-right
        ]
        
        mosaic_labels = []
        
        # Place each image in its quadrant
        for i, (img, img_labels) in enumerate(all_images[:4]):
            x1, y1, x2, y2 = positions[i]
            
            # Get placement position
            place_x1, place_y1 = 0, 0
            place_x2, place_y2 = x2 - x1, y2 - y1
            
            # Place image on mosaic
            mosaic_img[y1:y2, x1:x2] = cv2.resize(img, (place_x2, place_y2))
            
            # Adjust labels
            for label in img_labels:
                class_id, x_center, y_center, width, height = label
                
                # Scale and offset coordinates
                new_x_center = (x_center * place_x2 + place_x1 - place_x1) / (place_x2 - place_x1) * (x2 - x1) / w + x1 / w
                new_y_center = (y_center * place_y2 + place_y1 - place_y1) / (place_y2 - place_y1) * (y2 - y1) / h + y1 / h
                new_width = width * (place_x2 - place_x1) / w * (x2 - x1) / w
                new_height = height * (place_y2 - place_y1) / h * (y2 - y1) / h
                
                # Check if box is valid
                if 0 < new_x_center < 1 and 0 < new_y_center < 1 and new_width > 0 and new_height > 0:
                    mosaic_labels.append([class_id, new_x_center, new_y_center, new_width, new_height])
        
        return mosaic_img, np.array(mosaic_labels)
    
    def process_dataset(self, num_augmentations: int = 10) -> None:
        """Process the entire dataset to create augmented versions."""
        # Load all image-label pairs
        print("Loading dataset...")
        image_label_pairs = self.load_data()
        print(f"Found {len(image_label_pairs)} images with labels")
        
        # List of all augmentation methods with their probabilities
        augmentations = [
            (self.rotate, 0.7),
            (self.flip, 0.5),
            (self.scale, 0.6),
            (self.translate, 0.6),
            (self.crop_and_resize, 0.5),
            (self.adjust_brightness, 0.7),
            (self.adjust_contrast, 0.6),
            (self.color_jitter, 0.6),
            (self.add_noise, 0.4),
            (self.blur_sharpen, 0.4),
            (self.random_erase, 0.3),
            (self.mixup, 0.2),
            (self.mosaic, 0.1),
            (self.perspective_transform, 0.3)
        ]
        
        # Process each image
        for idx, (img_path, label_path) in enumerate(image_label_pairs):
            print(f"Processing image {idx+1}/{len(image_label_pairs)}: {os.path.basename(img_path)}")
            
            # Load image and labels
            image = cv2.imread(img_path)
            if image is None:
                print(f"Error: Could not load image {img_path}")
                continue
                
            labels = self.load_labels(label_path)
            
            # Copy original image and labels to target dir
            base_name = os.path.basename(img_path)
            label_name = os.path.basename(label_path)
            shutil.copy(img_path, os.path.join(self.target_images_dir, base_name))
            shutil.copy(label_path, os.path.join(self.target_labels_dir, label_name))
            
            # Create name for augmented images
            name_without_ext = os.path.splitext(base_name)[0]
            
            # Generate augmented versions
            for i in range(num_augmentations):
                # Start with original image and labels
                aug_image = image.copy()
                aug_labels = labels.copy()
                
                # Select 3-5 random augmentations
                num_augs = random.randint(3, 5)
                selected_augs = random.sample(augmentations, k=min(num_augs, len(augmentations)))
                
                # Apply selected augmentations
                for aug_func, prob in selected_augs:
                    if random.random() < prob:
                        aug_image, aug_labels = aug_func(aug_image, aug_labels)
                
                # Skip if no labels remain
                if len(aug_labels) == 0:
                    continue
                    
                # Save augmented image and labels
                aug_filename = f"{name_without_ext}_aug_{i+1}"
                cv2.imwrite(os.path.join(self.target_images_dir, f"{aug_filename}.jpg"), aug_image)
                self.save_labels(os.path.join(self.target_labels_dir, f"{aug_filename}.txt"), aug_labels)
        
        print(f"Augmentation completed. Files saved to {self.target_dir}")

In [ ]:
import os

# Define source and target directories
source_dir = "data/mountain/train"
target_dir = "data/mountain/augmented_train"

# Create data augmentation instance
augmentor = DataAugmentation(source_dir=source_dir, target_dir=target_dir)

# Process the dataset with 5 augmentations per image
# This will create 5 augmented versions of each original image
augmentor.process_dataset(num_augmentations=5)

# Print summary of files
original_images = len(os.listdir(os.path.join(source_dir, "images")))
augmented_images = len(os.listdir(os.path.join(target_dir, "images")))

print(f"Original dataset: {original_images} images")
print(f"Augmented dataset: {augmented_images} images")
print(f"Augmentation ratio: {augmented_images / original_images:.2f}x")